#### Packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans, AgglomerativeClustering

#### Load Data

In [2]:
# Data Path
file109 = r'C:\Users\mayda\Python practice\TIER\database\109_TESG變數彙總表.xlsx'
file109_T = r'C:\Users\mayda\Python practice\TIER\database\109_TESG永續發展指標主表.xlsx'
file110 = r'C:\Users\mayda\Python practice\TIER\database\110_TESG變數彙總表.xlsx'
file110_T = r'C:\Users\mayda\Python practice\TIER\database\110_TESG永續發展指標主表.xlsx'
file111 = r'C:\Users\mayda\Python practice\TIER\database\111_TESG變數彙總表.xlsx'
file111_T = r'C:\Users\mayda\Python practice\TIER\database\111_TESG永續發展指標主表.xlsx'
fileINFO = "database/上市公司基本資料.xlsx"

dfINFO = pd.read_excel(fileINFO)
df_109 = pd.read_excel(file109)
df_109T = pd.read_excel(file109_T)
df_110 = pd.read_excel(file110)
df_110T = pd.read_excel(file110_T)
df_111 = pd.read_excel(file111)
df_111T = pd.read_excel(file111_T)

c:\Users\mayda\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


##### 產業代碼對應

In [3]:
# 創一個全新的df
desc_columns = ["代號", "名稱","TESG評等季度", "TESG評等公告日", "採用永續報告書年度", "採用永續報告書是否驗證", "SASB主產業", "SASB子產業"]

df = df_110.copy()
for col in df.columns:
    if col not in desc_columns:
        df = df.drop(columns=[col], axis=1) 

id_columns = ['代號', '主計處產業名']
dfINFO = dfINFO[id_columns]
df = df.merge(dfINFO, on='代號', how='outer')
df = df.dropna(subset=['主計處產業名']) # 排除掉非製造業

In [11]:
print(len(df))

713


In [5]:
df.loc[df["主計處產業名"].str[0:2] == "08", "主計處產業名"] = 1
df.loc[df["主計處產業名"].str[0:2] == "09", "主計處產業名"] = 2
df.loc[df["主計處產業名"].str[0:2] == "10", "主計處產業名"] = 2
df.loc[df["主計處產業名"].str[0:2] == "11", "主計處產業名"] = 3
df.loc[df["主計處產業名"].str[0:2] == "12", "主計處產業名"] = 4
df.loc[df["主計處產業名"].str[0:2] == "13", "主計處產業名"] = 5
df.loc[df["主計處產業名"].str[0:2] == "14", "主計處產業名"] = 6
df.loc[df["主計處產業名"].str[0:2] == "15", "主計處產業名"] = 7
df.loc[df["主計處產業名"].str[0:2] == "16", "主計處產業名"] = 8
df.loc[df["主計處產業名"].str[0:2] == "17", "主計處產業名"] = 9
df.loc[df["主計處產業名"].str[0:2] == "18", "主計處產業名"] = 10
df.loc[df["主計處產業名"].str[0:2] == "19", "主計處產業名"] = 11
df.loc[df["主計處產業名"].str[0:2] == "20", "主計處產業名"] = 12
df.loc[df["主計處產業名"].str[0:2] == "21", "主計處產業名"] = 13
df.loc[df["主計處產業名"].str[0:2] == "22", "主計處產業名"] = 15
df.loc[df["主計處產業名"].str[0:2] == "23", "主計處產業名"] = 14
df.loc[df["主計處產業名"].str[0:2] == "24", "主計處產業名"] = 16
df.loc[df["主計處產業名"].str[0:2] == "25", "主計處產業名"] = 17
df.loc[df["主計處產業名"].str[0:2] == "26", "主計處產業名"] = 18
df.loc[df["主計處產業名"].str[0:2] == "27", "主計處產業名"] = 19
df.loc[df["主計處產業名"].str[0:2] == "28", "主計處產業名"] = 20
df.loc[df["主計處產業名"].str[0:2] == "29", "主計處產業名"] = 21
df.loc[df["主計處產業名"].str[0:2] == "30", "主計處產業名"] = 22
df.loc[df["主計處產業名"].str[0:2] == "31", "主計處產業名"] = 23
df.loc[df["主計處產業名"].str[0:2] == "32", "主計處產業名"] = 24
df.loc[df["主計處產業名"].str[0:2] == "33", "主計處產業名"] = 25
df.loc[df["主計處產業名"].str[0:2] == "34", "主計處產業名"] = 26

df.rename(columns={'主計處產業名': 'industry_id'}, inplace=True)

In [6]:
mask = df['industry_id'].str.len() >= 3
df = df[~mask]
print(df)

          代號   名稱 TESG評等季度   TESG評等公告日  採用永續報告書年度  採用永續報告書是否驗證  SASB主產業  \
14      1101   台泥  2021/12  2022/03/31     2020.0          1.0  提煉與礦產加工   
15      1102   亞泥  2021/12  2022/03/31     2020.0          1.0  提煉與礦產加工   
16      1103   嘉泥  2021/12  2022/03/31     2020.0          1.0  提煉與礦產加工   
17      1104   環泥  2021/12  2022/03/31     2020.0          0.0  提煉與礦產加工   
18      1108   幸福  2021/12  2022/03/31     2020.0          0.0  提煉與礦產加工   
...      ...  ...      ...         ...        ...          ...      ...   
2308    9105  NaN      NaN         NaN        NaN          NaN      NaN   
2310    9110  NaN      NaN         NaN        NaN          NaN      NaN   
2312  911622  NaN      NaN         NaN        NaN          NaN      NaN   
2313  911868  NaN      NaN         NaN        NaN          NaN      NaN   
2315    9136  NaN      NaN         NaN        NaN          NaN      NaN   

     SASB子產業 industry_id  
14      建築材料          14  
15      建築材料          14  
16      建築材料      

##### 計算年增率

In [18]:
def calculate_YoY(columns, df0, df1, df2, df):
    for column in columns:
        for index, row in df.iterrows():
            company = row['代號']  # Assuming '代號' is the column name for company identifier
            if company in df0.index and company in df1.index and company in df2.index:  # Check if company exists in all dataframes
                df.at[index, f"{column}_YoY"] = (df2.at[company, column] - df1.at[company, column]) / df1.at[company, column] # YoY calculation using df1 as base
                df.at[index, f"{column}_base_YoY"] = (df1.at[company, column] - df0.at[company, column]) / df0.at[company, column] # YoY calculation using df0 as base
            else:
                print(f"Company '{company}' is not present in all dataframes, skipping YoY calculation for this company in column '{column}'.")
    return df

columns_YoY = ["E1", "E7", "E9", "E10", "E15", "E16", "E17", "E20", "E21",
               "S1", "S2", "S4", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14", "S15", "S17", "S19", "S20", "S21", "S24", 
               "G1", "G5", "G6", "G7", "G8", "G13", "G14", "G19", "G20", "G21", "G22", "G24"]
calculate_YoY(columns_YoY, df_109, df_110, df_111, df)

Company '2301' is not present in all dataframes, skipping YoY calculation for this company in column 'E1'.
Company '2302' is not present in all dataframes, skipping YoY calculation for this company in column 'E1'.
Company '2303' is not present in all dataframes, skipping YoY calculation for this company in column 'E1'.
Company '2305' is not present in all dataframes, skipping YoY calculation for this company in column 'E1'.
Company '2308' is not present in all dataframes, skipping YoY calculation for this company in column 'E1'.
Company '2312' is not present in all dataframes, skipping YoY calculation for this company in column 'E1'.
Company '2313' is not present in all dataframes, skipping YoY calculation for this company in column 'E1'.
Company '2314' is not present in all dataframes, skipping YoY calculation for this company in column 'E1'.
Company '2316' is not present in all dataframes, skipping YoY calculation for this company in column 'E1'.
Company '2317' is not present in all 

<ipython-input-18-35d2b73ebc9e>:6: RuntimeWarning: invalid value encountered in scalar divide
  df.at[index, f"{column}_YoY"] = (df2.at[company, column] - df1.at[company, column]) / df1.at[company, column] # YoY calculation using df1 as base
<ipython-input-18-35d2b73ebc9e>:7: RuntimeWarning: invalid value encountered in scalar divide
  df.at[index, f"{column}_base_YoY"] = (df1.at[company, column] - df0.at[company, column]) / df0.at[company, column] # YoY calculation using df0 as base


Company '2436' is not present in all dataframes, skipping YoY calculation for this company in column 'E7'.
Company '2438' is not present in all dataframes, skipping YoY calculation for this company in column 'E7'.
Company '2439' is not present in all dataframes, skipping YoY calculation for this company in column 'E7'.
Company '2440' is not present in all dataframes, skipping YoY calculation for this company in column 'E7'.
Company '2441' is not present in all dataframes, skipping YoY calculation for this company in column 'E7'.
Company '2443' is not present in all dataframes, skipping YoY calculation for this company in column 'E7'.
Company '2449' is not present in all dataframes, skipping YoY calculation for this company in column 'E7'.
Company '2451' is not present in all dataframes, skipping YoY calculation for this company in column 'E7'.
Company '2454' is not present in all dataframes, skipping YoY calculation for this company in column 'E7'.
Company '2455' is not present in all 

<ipython-input-18-35d2b73ebc9e>:7: RuntimeWarning: divide by zero encountered in scalar divide
  df.at[index, f"{column}_base_YoY"] = (df1.at[company, column] - df0.at[company, column]) / df0.at[company, column] # YoY calculation using df0 as base
<ipython-input-18-35d2b73ebc9e>:6: RuntimeWarning: divide by zero encountered in scalar divide
  df.at[index, f"{column}_YoY"] = (df2.at[company, column] - df1.at[company, column]) / df1.at[company, column] # YoY calculation using df1 as base


Company '2305' is not present in all dataframes, skipping YoY calculation for this company in column 'E10'.
Company '2308' is not present in all dataframes, skipping YoY calculation for this company in column 'E10'.
Company '2312' is not present in all dataframes, skipping YoY calculation for this company in column 'E10'.
Company '2313' is not present in all dataframes, skipping YoY calculation for this company in column 'E10'.
Company '2314' is not present in all dataframes, skipping YoY calculation for this company in column 'E10'.
Company '2316' is not present in all dataframes, skipping YoY calculation for this company in column 'E10'.
Company '2317' is not present in all dataframes, skipping YoY calculation for this company in column 'E10'.
Company '2323' is not present in all dataframes, skipping YoY calculation for this company in column 'E10'.
Company '2324' is not present in all dataframes, skipping YoY calculation for this company in column 'E10'.
Company '2327' is not presen

,代號,名稱,TESG評等季度,TESG評等公告日,採用永續報告書年度,採用永續報告書是否驗證,SASB主產業,SASB子產業,industry_id,E1_YoY,...,S25_perf,G2_perf,G3_perf,G4_perf,E18_perf,LEVEL_110,TESG_110,LEVEL_111,TESG_111,TESG_change
0,1101,台泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,-0.144283,...,-10.0,1.0,-1.0,-1.0,NaN,B+,63.26,B+,61.79,-
1,1102,亞泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,-0.144283,...,-10.0,-1.0,0.0,0.0,NaN,A,67.84,A,64.49,-
2,1103,嘉泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,-0.252867,...,-1.0,1.0,0.0,0.0,NaN,B,56.31,B,56.67,-
3,1104,環泥,2021/12,2022/03/31,2020.0,0.0,提煉與礦產加工,建築材料,14,-0.675285,...,1.0,0.0,3.0,0.5,NaN,A,64.68,B+,61.72,↓
4,1108,幸福,2021/12,2022/03/31,2020.0,0.0,提煉與礦產加工,建築材料,14,1.851420,...,-6.0,-0.5,-3.0,-0.5,NaN,B+,58.28,B,52.82,↓
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9938,百和,2021/12,2022/03/31,2020.0,0.0,消費品,服飾、配件與鞋類,3,11.824852,...,0.0,0.0,0.0,0.0,NaN,B+,59.45,B+,59.60,-
709,9939,宏全,2021/12,2022/03/31,2020.0,1.0,資源轉化,容器與包裝,15,-0.412337,...,-9.0,0.0,1.0,0.0,NaN,B+,61.78,B+,60.01,-
710,9942,茂順,2021/12,2022/03/31,NaN,0.0,運輸,汽車零件,22,-0.882539,...,0.0,0.0,0.0,0.0,NaN,B+,57.72,B,54.06,↓
711,9944,新麗,2021/12,2022/03/31,NaN,0.0,消費品,服飾、配件與鞋類,3,-0.197061,...,0.0,0.0,0.0,0.0,NaN,B-,47.91,B-,50.48,-


- 計算增減

In [19]:
columns_YN = ["S22", "G9", "G10", "G11", "G12", "G15", "G16", "G17", "G18", "E19", "S26", "G23", "G25", "G26"]

def calculate_performance(col1, col2, col3, df0, df1, df):
    for col in df0.columns:
        if (col not in col1) & (col not in col2) & (col not in col3):
            df0.fillna(0)
            df1.fillna(0)          
            df[f"{col}_perf"] = df1[col]-df0[col]
    return df    
calculate_performance(columns_YoY, desc_columns, columns_YN, df_110, df_111, df)

,代號,名稱,TESG評等季度,TESG評等公告日,採用永續報告書年度,採用永續報告書是否驗證,SASB主產業,SASB子產業,industry_id,E1_YoY,...,S25_perf,G2_perf,G3_perf,G4_perf,E18_perf,LEVEL_110,TESG_110,LEVEL_111,TESG_111,TESG_change
0,1101,台泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,-0.144283,...,0.0,0.0,0.0,0.0,NaN,B+,63.26,B+,61.79,-
1,1102,亞泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,-0.144283,...,0.0,0.0,0.0,0.0,NaN,A,67.84,A,64.49,-
2,1103,嘉泥,2021/12,2022/03/31,2020.0,1.0,提煉與礦產加工,建築材料,14,-0.252867,...,0.0,0.0,0.0,0.0,NaN,B,56.31,B,56.67,-
3,1104,環泥,2021/12,2022/03/31,2020.0,0.0,提煉與礦產加工,建築材料,14,-0.675285,...,0.0,0.0,0.0,0.0,NaN,A,64.68,B+,61.72,↓
4,1108,幸福,2021/12,2022/03/31,2020.0,0.0,提煉與礦產加工,建築材料,14,1.851420,...,0.0,0.0,0.0,0.0,NaN,B+,58.28,B,52.82,↓
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9938,百和,2021/12,2022/03/31,2020.0,0.0,消費品,服飾、配件與鞋類,3,11.824852,...,-1.0,0.0,-1.0,0.0,NaN,B+,59.45,B+,59.60,-
709,9939,宏全,2021/12,2022/03/31,2020.0,1.0,資源轉化,容器與包裝,15,-0.412337,...,-9.5,0.0,-1.0,-0.5,NaN,B+,61.78,B+,60.01,-
710,9942,茂順,2021/12,2022/03/31,NaN,0.0,運輸,汽車零件,22,-0.882539,...,-4.0,0.5,3.5,0.5,NaN,B+,57.72,B,54.06,↓
711,9944,新麗,2021/12,2022/03/31,NaN,0.0,消費品,服飾、配件與鞋類,3,-0.197061,...,5.0,1.0,2.0,1.0,NaN,B-,47.91,B-,50.48,-


- 合併最終成績

In [20]:
left = df
right = df_110T[["代號", "TESG等級", "TESG分數"]]
df = pd.merge(left, right, on="代號")
df.rename(columns = {'TESG等級': 'LEVEL_110'}, inplace=True)
df.rename(columns = {'TESG分數': 'TESG_110'}, inplace=True)

left = df
right = df_111T[["代號", "TESG等級", "TESG分數", "TESG升降等"]]
df = pd.merge(left, right, on="代號")
df.rename(columns = {'TESG等級': 'LEVEL_111'}, inplace=True)
df.rename(columns = {'TESG分數': 'TESG_111'}, inplace=True)
df.rename(columns = {'TESG升降等': 'TESG_change'}, inplace=True)

##### 輸出成csv

In [22]:
# 變成CSV檔
df.to_csv("database/ESG_TESG.csv", index=False, encoding='utf-8-sig')